In [3]:
import numpy as np
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, BatchNormalization
from keras.layers import Dropout
from keras.layers import Input, concatenate, UpSampling2D, Reshape, core

from tensorflow.keras.utils import to_categorical
from keras import regularizers
import os
import matplotlib.pyplot as plt


In [5]:
import cv2
def load_images_from_folder(path):
  i=0
  image = []
  mask = []
  files= os.listdir(path) #得到資料夾下的所有檔名稱
  Resize= (512,512)
  for root, dirs, files in os.walk(path):
    for file in files:
      fullpath = os.path.join(root, file)
      img = np.array(cv2.imread(fullpath)) 
      if file=="img.png":
        img = cv2.resize(img, Resize, interpolation=cv2.INTER_AREA)
        image.append(img)
      elif file=="label.png":
        img = cv2.resize(img, Resize, interpolation=cv2.INTER_AREA)
        mask.append(img)
      i+=1
      if(i%50==0):
        print(i)
  image = np.array(image).astype("float32")/255
  mask = np.array(mask).astype("float32")/255
  return image, mask

In [7]:
def unet(Inshape):
    from keras import losses
    inputs = Input(shape=Inshape)
    conv1 = Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    #512
    conv2 = Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    #256
    conv3 = Conv2D(256, (3, 3), activation = 'relu', padding = 'same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, (3, 3), activation = 'relu', padding = 'same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    #128
    conv4 = Conv2D(512, (3, 3), activation = 'relu', padding = 'same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(512, (3, 3), activation = 'relu', padding = 'same')(conv4)
    conv4 = BatchNormalization()(conv4)
    drop4 = Dropout(0.2)(conv4)
    #128
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    #64
    conv5 = Conv2D(1024, (3, 3), activation = 'relu', padding = 'same')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(1024, (3, 3), activation = 'relu', padding = 'same')(conv5)
    conv5 = BatchNormalization()(conv5)
    drop5 = Dropout(0.2)(conv5)
    #64
    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, (3, 3), activation = 'relu', padding = 'same')(merge6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(512, (3, 3), activation = 'relu', padding = 'same')(conv6)
    conv6 = BatchNormalization()(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, (3, 3), activation = 'relu', padding = 'same')(merge7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(256, (3, 3), activation = 'relu', padding = 'same')(conv7)
    conv7 = BatchNormalization()(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(merge8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(conv8)
    conv8 = BatchNormalization()(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(merge9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(conv9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(3, (3, 3), activation = 'relu', padding = 'same')(conv9)
    conv9 = BatchNormalization()(conv9)
    conv10 = core.Activation('softmax')(conv9)
    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer=SGD(learning_rate=0.01, momentum= 0.9), loss="categorical_crossentropy",metrics=['accuracy'])
    return model
    

In [8]:
dir_train = "./EKG_seg/train"#training
img, mask = load_images_from_folder(dir_train)
dir_validation ="./EKG_seg/validation"
V_img, V_mask =  load_images_from_folder(dir_validation)



50
100
150
200
250
300
350
400
450
500
550
600
650
700
50
100
150
200


In [9]:
from tensorflow.keras.optimizers import SGD

IN = (img.shape[1],img.shape[2],img.shape[3])
model = unet(IN)
history = model.fit(img, mask,  epochs= 30, batch_size=4,validation_data = (V_img, V_mask))


Epoch 1/30
44/44 [==============================] - 248s 5s/step - loss: 0.0728 - accuracy: 0.3290 - val_loss: 0.0614 - val_accuracy: 0.0949
Epoch 2/30
44/44 [==============================] - 171s 4s/step - loss: 0.0648 - accuracy: 0.1727 - val_loss: 0.0619 - val_accuracy: 0.0288
Epoch 3/30
44/44 [==============================] - 170s 4s/step - loss: 0.0600 - accuracy: 0.2215 - val_loss: 0.0622 - val_accuracy: 0.0288
Epoch 4/30
44/44 [==============================] - 170s 4s/step - loss: 0.0565 - accuracy: 0.3099 - val_loss: 0.0622 - val_accuracy: 0.0288
Epoch 5/30
44/44 [==============================] - 171s 4s/step - loss: 0.0539 - accuracy: 0.2506 - val_loss: 0.0622 - val_accuracy: 0.0288
Epoch 6/30
44/44 [==============================] - 170s 4s/step - loss: 0.0502 - accuracy: 0.2383 - val_loss: 0.0622 - val_accuracy: 0.0289
Epoch 7/30
44/44 [==============================] - 170s 4s/step - loss: 0.0489 - accuracy: 0.2539 - val_loss: 0.0621 - val_accuracy: 0.0310
Epoch 8/30
44

In [16]:
model.fit(img, mask,  epochs= 10, batch_size=4,validation_data = (V_img, V_mask))

Epoch 1/10
44/44 [==============================] - 175s 4s/step - loss: 0.0427 - accuracy: 0.3485 - val_loss: 0.0423 - val_accuracy: 0.1007
Epoch 2/10
44/44 [==============================] - 176s 4s/step - loss: 0.0413 - accuracy: 0.3945 - val_loss: 0.0406 - val_accuracy: 0.1035
Epoch 3/10
44/44 [==============================] - 176s 4s/step - loss: 0.0434 - accuracy: 0.4013 - val_loss: 0.0344 - val_accuracy: 0.1280
Epoch 4/10
44/44 [==============================] - 176s 4s/step - loss: 0.0431 - accuracy: 0.3299 - val_loss: 0.0377 - val_accuracy: 0.1098
Epoch 5/10
44/44 [==============================] - 176s 4s/step - loss: 0.0438 - accuracy: 0.3900 - val_loss: 0.0385 - val_accuracy: 0.1146
Epoch 6/10
44/44 [==============================] - 176s 4s/step - loss: 0.0434 - accuracy: 0.4070 - val_loss: 0.0404 - val_accuracy: 0.1034
Epoch 7/10
44/44 [==============================] - 176s 4s/step - loss: 0.0426 - accuracy: 0.3535 - val_loss: 0.0401 - val_accuracy: 0.1049
Epoch 8/10
44

In [ ]:
model.save("./model1.h5")